In [133]:
import pandas as pd

In [134]:
%cd ../datasets/

c:\Personal Files\Arya\Programs\FloatChat\datasets


In [135]:
#Cleaning the dataset 2002
df=pd.read_csv(f'ARGO_OBSERVATIONS_ids_info_table.csv')

In [136]:
df

,wmo_id,first_obs_date,latitude,longitude,no_of_profiles,status,deployed_country
0,19001,2003-07-10 13:56:00,-16.871,58.622,10,INACTIVE,OTHERS
1,53546,2001-01-07 10:52:32,-14.158,120.410,42,INACTIVE,Australia
2,53547,2001-01-10 02:12:27,-7.419,101.661,89,INACTIVE,Australia
3,53548,2001-01-01 00:37:41,-15.252,114.304,120,INACTIVE,Australia
4,53553,2001-01-03 18:12:41,-14.046,107.970,56,INACTIVE,Australia
...,...,...,...,...,...,...,...
2858,6900746,2012-01-17 05:35:32,-37.209,20.185,64,INACTIVE,OTHERS
2859,6900747,2011-05-14 17:31:08,-47.176,20.267,76,INACTIVE,OTHERS
2860,6900748,2011-08-24 05:19:11,-48.286,20.790,85,INACTIVE,OTHERS
2861,6900749,2011-07-16 10:38:09,-50.904,20.392,91,INACTIVE,OTHERS


In [137]:
df.info() #Info about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2863 entries, 0 to 2862
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wmo_id            2863 non-null   int64  
 1   first_obs_date    2863 non-null   object 
 2   latitude          2863 non-null   float64
 3   longitude         2863 non-null   float64
 4   no_of_profiles    2863 non-null   int64  
 5   status            2863 non-null   object 
 6   deployed_country  2863 non-null   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 156.7+ KB


In [138]:
df.isnull().sum() ##No null values - checked

wmo_id              0
first_obs_date      0
latitude            0
longitude           0
no_of_profiles      0
status              0
deployed_country    0
dtype: int64

In [139]:
# Set threshold (e.g., drop if more than 90% values are null)
threshold = 0.3

# Calculate null ratio per column
null_ratio = df.isna().sum() / len(df)

# Columns where null ratio exceeds threshold
cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()

# Drop those columns
df.drop(columns=cols_to_drop, inplace=True)

# For the remaining columns, drop null rows and reset index
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [140]:
df.columns #Checking if there's extra whitespace in columns

Index(['wmo_id', 'first_obs_date', 'latitude', 'longitude', 'no_of_profiles',
       'status', 'deployed_country'],
      dtype='object')

In [141]:
#Trim if extra whitespace
df.columns = df.columns.str.strip()

In [142]:
# Parse mixed datetime formats safely
df["first_obs_date"] = pd.to_datetime(df["first_obs_date"], dayfirst=True, format='mixed')

# Split into separate columns
df["first_observation_date"] = df["first_obs_date"].dt.date
df["first_observation_time"] = df["first_obs_date"].dt.time

df.drop('first_obs_date',axis=1,inplace=True)

In [143]:
#seeing new info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2863 entries, 0 to 2862
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   wmo_id                  2863 non-null   int64  
 1   latitude                2863 non-null   float64
 2   longitude               2863 non-null   float64
 3   no_of_profiles          2863 non-null   int64  
 4   status                  2863 non-null   object 
 5   deployed_country        2863 non-null   object 
 6   first_observation_date  2863 non-null   object 
 7   first_observation_time  2863 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 179.1+ KB


In [144]:
df.head()

,wmo_id,latitude,longitude,no_of_profiles,status,deployed_country,first_observation_date,first_observation_time
0,19001,-16.871,58.622,10,INACTIVE,OTHERS,2003-07-10,13:56:00
1,53546,-14.158,120.410,42,INACTIVE,Australia,2001-01-07,10:52:32
2,53547,-7.419,101.661,89,INACTIVE,Australia,2001-01-10,02:12:27
3,53548,-15.252,114.304,120,INACTIVE,Australia,2001-01-01,00:37:41
4,53553,-14.046,107.970,56,INACTIVE,Australia,2001-01-03,18:12:41


In [145]:
#Checking for duplicate tuples
df.duplicated().value_counts()

False    2863
Name: count, dtype: int64

In [146]:
df[df.duplicated(keep=False)] #Seeing the duplicated rows

,wmo_id,latitude,longitude,no_of_profiles,status,deployed_country,first_observation_date,first_observation_time


In [147]:
# Drop duplicate rows
df = df.drop_duplicates()

# Reset the index so it’s clean and sequential
df = df.reset_index(drop=True)

In [148]:
#Final Verification
df.duplicated().value_counts()

False    2863
Name: count, dtype: int64

In [149]:
df.to_csv(f'cleaned/ARGO_DB_ids_info_table_cleaned.csv',index=False)